In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn import preprocessing
import statsmodels.api as sm
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
import patsy
%matplotlib inline

In [2]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [3]:
df = pd.read_csv("Data/county_data_clean.csv")

#### Dataframe for State and Counties

In [4]:
df_states = df[df["county_fips_code"] == 0] #Dataframe for states only
df_counties = df[df["county_fips_code"] != 0] #Dataframe for counties only

In [5]:
df_counties = df_counties.set_index("name")
df_states = df_states.set_index("name")

In [6]:
df_counties = df_counties.drop(["Unnamed: 0"], axis=1)
df_states = df_states.drop(["Unnamed: 0"], axis=1)

In [7]:
df_counties["insured_adults_raw"] = df_counties["adult_population_18_64"] - df_counties["uninsured_adults_raw"]

# Logistic Regression

#### Create dataframe with counties in IL only.

In [8]:
df_IL = df_counties[df_counties["state_abbreviation"] == "IL"]

#### Drop completely missing columns.

In [9]:
df_IL = df_IL.dropna(axis = 1, how = "all") 

####  Dropping geographic information that will not help with inference or prediction.

In [10]:
df_IL = df_IL.drop(["state_fips_code", "state_abbreviation", "county_fips_code", "5_digit_fips_code"], axis = 1)

#### Dimensions of data

In [11]:
df_IL.shape #102 counties, 118 columns

(102, 121)

#### Missing values by column.

In [12]:
df_IL.isna().mean().sort_values(ascending = True)

premature_death                                                         0.000000
frequent_mental_distress                                                0.000000
frequent_physical_distress                                              0.000000
premature_age_adjusted_mortality                                        0.000000
life_expectancy                                                         0.000000
long_commute_driving_alone                                              0.000000
p_rural                                                                 0.000000
percentage_of_households_with_lack_of_kitchen_or_plumbing_facilities    0.000000
percentage_of_households_with_overcrowding                              0.000000
percentage_of_households_with_high_housing_costs                        0.000000
severe_housing_problems                                                 0.000000
drinking_water_violations                                               0.000000
air_pollution_particulate_ma

In [13]:
df_IL = df_IL.drop(["infant_mortality_hispanic", "infant_mortality_white", "infant_mortality_black",
                   "child_mortality_hispanic", "child_mortality_white", "child_mortality_black"], axis = 1)

In [14]:
df_IL.isna().mean().sort_values(ascending = True)

premature_death                                                         0.000000
frequent_mental_distress                                                0.000000
frequent_physical_distress                                              0.000000
premature_age_adjusted_mortality                                        0.000000
life_expectancy                                                         0.000000
long_commute_driving_alone                                              0.000000
driving_alone_to_work                                                   0.000000
percentage_of_households_with_lack_of_kitchen_or_plumbing_facilities    0.000000
percentage_of_households_with_overcrowding                              0.000000
p_rural                                                                 0.000000
severe_housing_problems                                                 0.000000
drinking_water_violations                                               0.000000
air_pollution_particulate_ma

#### Missing values by county.

In [15]:
df_IL.isna().mean(axis = 1).sort_values(ascending = True)

name
Kane County           0.000000
McHenry County        0.000000
Peoria County         0.000000
St. Clair County      0.000000
Will County           0.000000
Winnebago County      0.000000
DuPage County         0.000000
Lake County           0.000000
DeKalb County         0.000000
Champaign County      0.000000
Vermilion County      0.000000
Kankakee County       0.000000
Rock Island County    0.000000
Madison County        0.000000
Cook County           0.000000
Kendall County        0.008696
McLean County         0.017391
Knox County           0.017391
La Salle County       0.017391
Stephenson County     0.026087
Sangamon County       0.026087
Macon County          0.026087
Jackson County        0.034783
Williamson County     0.043478
Jefferson County      0.060870
Boone County          0.060870
Marion County         0.069565
Adams County          0.069565
Whiteside County      0.069565
Tazewell County       0.069565
Coles County          0.095652
Grundy County         0.095652
Lee

#### Labels and Features

In [16]:
labels = df_IL[['uninsured_adults_raw','insured_adults_raw']]

In [17]:
features = df_IL.drop(["uninsured_adults",  
                      "uninsured_children", 
                      "uninsured",
                      "uninsured_adults_raw",
                      "insured_adults_raw",
                      "population",
                      "adult_population_18_64"], axis = 1)

In [18]:
feature_list = list(features.columns) 

#### Splitting into train and test set

In [19]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.3, random_state = 123)

In [20]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (71, 108)
Training Labels Shape: (71, 2)
Testing Features Shape: (31, 108)
Testing Labels Shape: (31, 2)


#### Imputing Missing Values

In [21]:
imp = IterativeImputer(max_iter=50, random_state=123)
imp.fit(train_features)
train_features = imp.transform(train_features)

C:\Users\bhatt\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:669: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


In [22]:
imp.fit(test_features)
test_features = imp.transform(test_features)

#### Standardizing Data

In [23]:
scaler = preprocessing.StandardScaler()
scaler.fit(train_features) 
train_features = scaler.transform(train_features)

In [24]:
scaler.fit(test_features)
test_features = scaler.transform(test_features)

#### Recursive Feature Elimination

In [25]:
rfe = RFE(estimator=RandomForestRegressor(), n_features_to_select=20)
fit = rfe.fit(train_features, train_labels) 

In [26]:
print("Num Features: %s" % (fit.n_features_))
print("Selected Features: %s" % (fit.support_))
print("Feature Ranking: %s" % (fit.ranking_))

Num Features: 20
Selected Features: [False False False False False False False False False False False False
 False False False  True False False False False False False  True False
 False  True  True False False False  True False False False False False
 False False False False False False False False False False False False
 False False  True False False  True False  True  True False False  True
 False False False False False False False False False False False False
 False False False False False  True False False False  True  True False
 False False False False False False False  True False False False False
  True False False False False  True False  True  True  True False  True]
Feature Ranking: [28 73  3 60 26 36 24 32 61 23 33 87  8 71 52  1 18  9  6 80 50 63  1 75
 14  1  1 20 78 56  1 74 89 37 81 22 59 55 35  5 12 31 70 34 41 17 79  4
 76 49  1 54 19  1 88  1  1  7 48  1 86 45 64 27 44 83 42 68 77 30 67 25
 53 57 40 46 62  1 72 39 16  1  1 10 13 47 65 66 85  2 69  1 51 21 58 

In [27]:
indices = [i for i, x in enumerate(fit.support_) if x]
selected_features = pd.Series(feature_list)[indices]

In [28]:
train_features[:, indices]

array([[ 0.48698764,  1.61951695, -0.27542877, ..., -0.85696328,
        -0.18841604, -0.75002853],
       [-0.34415595, -0.06226702, -0.34335324, ...,  0.76563787,
        -0.47526955,  0.70731397],
       [-0.16859071, -0.86427192,  0.14567464, ..., -0.0865442 ,
         1.0596419 , -0.16870017],
       ...,
       [ 1.60535245, -1.82589692,  0.7268753 , ..., -2.17098071,
         1.63191647, -1.75270822],
       [ 0.30048761, -0.12148035,  0.16745176, ...,  0.59604898,
        -0.28285137, -0.46968588],
       [ 0.63285422, -1.41651569,  2.09598387, ...,  0.79212293,
        -0.39250046, -0.37930773]])

#### Fitting Model

In [29]:
train_set = pd.DataFrame(train_features[:, indices], columns = selected_features)
train_labels2 = train_labels.reset_index(drop = True)
train_set["uninsured_adults_raw"] = train_labels2["uninsured_adults_raw"]
train_set["insured_adults_raw"] = train_labels2["insured_adults_raw"]

In [30]:
train_set.head()

,access_to_exercise_opportunities,teen_births_white,dentists,ratio_of_population_to_dentists,preventable_hospital_stays_black,social_associations,air_pollution_particulate_matter,severe_housing_problems,percentage_of_households_with_high_housing_costs,driving_alone_to_work,hiv_prevalence,motor_vehicle_crash_deaths,insufficient_sleep,residential_segregation_black_white,severe_housing_cost_burden,p_asian,p_hispanic,p_non_hispanic_white,p_not_proficient_in_english,p_rural,uninsured_adults_raw,insured_adults_raw
0,0.486988,1.619517,-0.275429,-0.176385,0.604274,-0.373190,0.100373,0.123037,0.260813,0.656348,0.874028,-0.562942,1.204883,1.044183,0.270783,-0.153712,-0.031995,-0.856963,-0.188416,-0.750029,3201.0,40117.0
1,-0.344156,-0.062267,-0.343353,-0.114615,-0.016164,0.170105,-1.139011,-1.247001,-1.171898,0.264623,-0.883192,0.285438,-0.860315,0.534353,-1.465502,-0.440697,-0.598189,0.765638,-0.475270,0.707314,651.0,9579.0
2,-0.168591,-0.864272,0.145675,-0.472336,0.439370,-0.389809,-0.054550,0.175235,0.239116,-0.316921,-0.439201,-0.579413,-0.882345,0.407563,0.559667,-0.342343,0.789188,-0.086544,1.059642,-0.168700,2097.0,28244.0
3,-2.670324,0.823270,-1.008085,0.933280,-0.249016,-0.179204,-1.448856,-1.033353,-0.937724,-0.279901,0.193976,1.173493,-0.605259,-0.769678,-0.783888,-0.582268,-0.632784,0.848488,-0.448817,1.772310,202.0,2796.0
4,0.109529,0.687532,-0.750091,0.394958,-0.254192,1.223346,0.255295,-0.490243,-0.848254,0.081057,-0.804703,0.187210,-0.014380,0.096348,-0.957578,-0.159077,-0.542876,0.670441,-0.397099,-0.301996,635.0,8431.0


In [31]:
def formula_from_cols(features):
    return ' ~ ' + ' + '.join([col for col in features])

In [32]:
glm_formula = 'uninsured_adults_raw + insured_adults_raw' + formula_from_cols(selected_features)

In [33]:
glm_formula

'uninsured_adults_raw + insured_adults_raw ~ access_to_exercise_opportunities + teen_births_white + dentists + ratio_of_population_to_dentists + preventable_hospital_stays_black + social_associations + air_pollution_particulate_matter + severe_housing_problems + percentage_of_households_with_high_housing_costs + driving_alone_to_work + hiv_prevalence + motor_vehicle_crash_deaths + insufficient_sleep + residential_segregation_black_white + severe_housing_cost_burden + p_asian + p_hispanic + p_non_hispanic_white + p_not_proficient_in_english + p_rural'

In [34]:
binom_glm = smf.glm(formula = glm_formula, data = train_set, family = sm.families.Binomial())

In [35]:
binom_results = binom_glm.fit()

In [36]:
binom_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               Generalized Linear Model Regression Results                                
==========================================================================================================
Dep. Variable:     ['uninsured_adults_raw', 'insured_adults_raw']   No. Observations:                   71
Model:                                                        GLM   Df Residuals:                       50
Model Family:                                            Binomial   Df Model:                           20
Link Function:                                              logit   Scale:                          1.0000
Method:                                                      IRLS   Log-Likelihood:                -859.89
Date:                                            Fri, 30 Jul 2021   Deviance:                       1084.7
Time:                                                    16:52:11   Pearson chi2:                 1.10e+03
No. Iterations:                                                 6                                         
Covariance Type:                                        nonrobust                                         
====================================================================================================================
                                                       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                           -2.5573      0.004   -611.147      0.000      -2.565      -2.549
access_to_exercise_opportunities                    -0.0040      0.009     -0.431      0.666      -0.022       0.014
teen_births_white                                    0.0854      0.007     13.085      0.000       0.073       0.098
dentists                                            -0.0065      0.005     -1.256      0.209      -0.017       0.004
ratio_of_population_to_dentists                     -0.0083      0.009     -0.959      0.337      -0.025       0.009
preventable_hospital_stays_black                     0.0110      0.004      2.669      0.008       0.003       0.019
social_associations                                 -0.0115      0.007     -1.674      0.094      -0.025       0.002
air_pollution_particulate_matter                    -0.0178      0.004     -4.619      0.000      -0.025      -0.010
severe_housing_problems                              0.0887      0.021      4.174      0.000       0.047       0.130
percentage_of_households_with_high_housing_costs    -0.0267      0.023     -1.175      0.240      -0.071       0.018
driving_alone_to_work                               -0.0448      0.005     -8.818      0.000      -0.055      -0.035
hiv_prevalence                                      -0.0350      0.006     -6.265      0.000      -0.046      -0.024
motor_vehicle_crash_deaths                           0.0138      0.007      2.108      0.035       0.001       0.027
insufficient_sleep                                   0.0416      0.006      7.087      0.000       0.030       0.053
residential_segregation_black_white                  0.0220      0.005      4.853      0.000       0.013       0.031
severe_housing_cost_burden                           0.0189      0.011      1.711      0.087      -0.003       0.040
p_asian                                             -0.0124      0.004     -2.861      0.004      -0.021      -0.004
p_hispanic                                           0.0921      0.016      5.757      0.000       0.061       0.123
p_non_hispanic_white                                 0.0421      0.013      3.264      0.001       0.017       0.067
p_not_proficient_in_english                          0.0786      0.014      5.420      0.000       0.050       0.107
p_rural                                              0.0545      0.010      5.342   

In [52]:
actual = train_labels["uninsured_adults_raw"] / ( train_labels["uninsured_adults_raw"] +  train_labels["insured_adults_raw"])

In [62]:
binom_results.fittedvalues

0     0.076764
1     0.065131
2     0.078986
3     0.075311
4     0.069362
5     0.068102
6     0.067407
7     0.070578
8     0.073114
9     0.080740
10    0.081517
11    0.091900
12    0.062179
13    0.091711
14    0.070571
15    0.064474
16    0.071729
17    0.071373
18    0.081047
19    0.072647
20    0.067197
21    0.067023
22    0.074953
23    0.071650
24    0.072844
25    0.081035
26    0.119668
27    0.077807
28    0.074051
29    0.117946
30    0.077894
31    0.077907
32    0.060540
33    0.071064
34    0.067078
35    0.073396
36    0.070867
37    0.061588
38    0.069005
39    0.081217
40    0.082526
41    0.077793
42    0.078587
43    0.064663
44    0.067855
45    0.090275
46    0.069983
47    0.067435
48    0.050717
49    0.071392
50    0.081438
51    0.059413
52    0.070141
53    0.075274
54    0.069702
55    0.080682
56    0.064973
57    0.070007
58    0.075100
59    0.072758
60    0.071187
61    0.070452
62    0.051417
63    0.065799
64    0.060076
65    0.062569
66    0.06

In [63]:
actual

name
Vermilion County      0.073895
Hancock County        0.063636
Ogle County           0.069114
Scott County          0.067378
Richland County       0.070042
White County          0.076002
Williamson County     0.069870
Mason County          0.073574
Clay County           0.071102
Douglas County        0.109656
Union County          0.081977
Iroquois County       0.085592
Madison County        0.062856
Alexander County      0.075041
Pope County           0.082334
Clark County          0.060823
Wayne County          0.079743
Coles County          0.081336
McDonough County      0.076072
Crawford County       0.068813
Ford County           0.070380
Lee County            0.062644
Kankakee County       0.082676
Calhoun County        0.063977
Pike County           0.093814
Greene County         0.075996
Kane County           0.119893
Warren County         0.092001
Johnson County        0.068084
Cook County           0.117878
Stark County          0.064454
DeKalb County         0.081740
Cli

NameError: name 'mean' is not defined